# 3.2
## 3.2.1

In [1]:
# python list
a = [1.0, 2.0, 1.0]

In [2]:
a[0]

1.0

In [4]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

## 3.3.2

In [5]:
import torch
# 長さ3全ての要素1のテンソル
a = torch.ones(3)

In [6]:
a

tensor([1., 1., 1.])

In [7]:
float(a[1])

1.0

In [8]:
# floatに変換しないとtensor型のまま残る
a[1]

tensor(1.)

In [9]:
# setterは普通
a[2] = 2.0
a

tensor([1., 1., 2.])

## 3.2.3

In [10]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [18]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [15]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [20]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [21]:
points.shape

torch.Size([3, 2])

In [22]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [23]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points[0, 1]

tensor(1.)

In [24]:
points[0]

tensor([4., 1.])

In [26]:
some_list = list(range(6))
print(some_list[:])
print(some_list[1:4])
print(some_list[1:])
print(some_list[:4])
print(some_list[:-1])
print(some_list[1:4:2])

[0, 1, 2, 3, 4, 5]
[1, 2, 3]
[1, 2, 3, 4, 5]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 3]


In [30]:
print(points[1:])
print(points[1:,:])
print(points[1:, 0]) # 基本的に全部[行,列]で表していけば大丈夫ってことがわかった
print(points[None]) # 新たな次元を追加(unsqueezeでも可能)

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


# 3.4

In [32]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [33]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]
# 後ろからcolumns, rows, channels, batchの順番を守る

In [34]:
img_gray_naive = img_t.mean(-3)
# deepの世界では後ろから3番目はcolorっていうのが絶対のきまり
# 可読性とか考えると微妙な感じもあるが、計算速度とのトレードオフになってしまうので、
# これはルールとしている
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [35]:
print(img_gray_naive)
print(batch_gray_naive)

tensor([[-0.2806,  0.8820, -0.1662, -0.9802, -0.0724],
        [-0.0342,  0.5567,  0.2309,  0.2560,  0.2127],
        [ 0.0494, -0.6430, -0.4882,  0.3272, -0.7671],
        [-0.1786,  0.0324, -0.1286,  0.4404,  0.3282],
        [-0.7233, -0.3071,  0.2049, -0.5127, -0.6106]])
tensor([[[ 0.7852,  1.2976, -0.5324, -0.7889, -0.7645],
         [ 1.1792,  0.0820, -0.4632,  0.8688, -0.3243],
         [ 0.4507,  0.2715,  1.0412,  0.1579, -0.2435],
         [ 0.4605, -0.3891, -0.2158, -0.2255, -0.9145],
         [-0.1328, -0.3570,  0.5650,  0.6591, -0.6625]],

        [[-0.0472, -0.8315, -0.3356, -1.3460,  0.4304],
         [ 0.5673, -0.2249,  0.4160,  0.4490,  0.8557],
         [-0.6496,  0.6855, -0.7965,  0.2772, -0.7572],
         [-0.1261, -0.4751,  0.1217, -1.0922, -0.2899],
         [-0.7912, -0.1222,  0.4052, -0.3405, -0.0466]]])


In [36]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
img_weights = (img_t * unsqueezed_weights) # アダマール積
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [37]:
# 総和とかをうまく記号で書きやすくしてる
img_gray_weighted_fancy = torch.einsum("...chw,c->...hw", img_t, weights)
batch_gray_ewighted_fancy = torch.einsum("...chw,c->...hw", batch_t, weights)
batch_gray_ewighted_fancy.shape

torch.Size([2, 5, 5])

In [38]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=["channels"])
weights_named

/Users/fukuitomohiro/.pyenv/versions/3.6.8/envs/pytorch_basic/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1408.)
  """Entry point for launching an IPython kernel.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [39]:
img_named = img_t.refine_names(..., "channels", "rows", "columns")
batch_named = batch_t.refine_names(..., "channels", "rows", "columns")
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [41]:
# 名前に基づいて自動で次元を整理してくれる
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [42]:
gray_named = (img_named * weights_aligned).sum("channels")
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [43]:
gray_named = (img_named[..., :3] * weights_named).sum("channels")

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [44]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

# 3.5
## 3.5.3

In [45]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [46]:
short_points.dtype

torch.int16

In [47]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [48]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

In [49]:
points_64 = torch.rand(5, dtype=torch.double) # randは0-1の乱数を作成
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

# 3.6

In [52]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1) # 2軸の入れ替えをしている

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [53]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

# 3.7
## 3.7.1

In [54]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [55]:
points_storage = points.storage()
points_storage[0]

4.0

In [56]:
points.storage()[1]

1.0

In [57]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

## 3.7.2

In [59]:
a = torch.ones(3, 2)

In [60]:
# メソッド名の最後が_なものはimplace操作
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

# 3.8
## 3.8.1

In [61]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [62]:
second_point.size()

torch.Size([2])

In [63]:
second_point.shape

torch.Size([2])

In [64]:
points.stride()

(2, 1)

In [65]:
# 上記を利用して、
# storage_offset + stride[0] * i + storide[1] * j
# を利用してアクセス

In [66]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [67]:
second_point.storage_offset()

2

In [68]:
second_point.stride()

(1,)

In [69]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [71]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone() # python のdeepcopyみたいなもんだな
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

## 3.8.1

In [72]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [73]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [74]:
id(points.storage()) == id(points_t.storage())

True

In [75]:
points.stride()

(2, 1)

In [76]:
points_t.stride()

(1, 2)

## 3.8.3

In [77]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [78]:
transpose_t.shape

torch.Size([5, 4, 3])

In [79]:
some_t.stride()

(20, 5, 1)

In [80]:
transpose_t.stride()

(1, 5, 20)

## 3.8.4

In [81]:
points.is_contiguous()

True

In [82]:
points_t.is_contiguous()

False

In [83]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [84]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [85]:
points_t.stride()

(1, 2)

In [86]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [87]:
points_t_cont.stride()

(3, 1)

In [88]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]